## Neural Network

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm

In [ ]:
#Amount of features inputted in NN
NUMBER_FEATURES = 5

#Gaussian Noise
USE_GAUSSIAN_NOISE = False
VALUE_GAUSSIAN_NOISE = 70 # Only needed when Gaussian noise = True


config = dict(NUMBER_FEATURES = NUMBER_FEATUERS, USE_GAUSSIAN_NOISE = USE_GAUSSIAN_NOISE, 
              VALUE_GAUSSIAN_NOISE = VALUE_GAUSSIAN_NOISE)

In [ ]:
use_gaussian_noise = config["USE_GAUSSIAN_NOISE"]
value_gaussian_noise = config["VALUE_GAUSSIAN_NOISE"]

if use_gaussian_noise:
    inp = tf.keras.layers.GaussianNoise(value_gaussian_noise)(inp)



In [5]:
def build_model(config):
    size = config["NUMBER_FEATURES"]

    inp = tf.keras.layers.Input(shape=size, name = "input_features")
    
    inp2 = tf.keras.layers.Input(shape = 2, name = "FVC_Start & Weeks_from_start")
    
    inputs = [inp, inp2]
    outputs = [inp2]
    
    x = tf.keras.layers.Dense(10, activation='relu')(inp)
    x = tf.keras.layers.Dense(10, activation='relu')(x)
    
    # output : [slope, s, FVC_start, weeks_from_start]
    outputs += tf.keras.layers.Dense(2, name = "Output_a_s")(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    
    def Laplace_log_likelihood(y_true, y_pred):
        # y_pred = [slope, s, FVC_start, weeks_from_start]
        tf.dtypes.cast(y_true, tf.float32)
        tf.dtypes.cast(y_pred, tf.float32)
        
        slope = y_pred[:,0]
        s = y_pred[:,1]
        FVC_start = y_pred[:,2]
        weeks_from_start = y_pred[:,3]
        
        sigma = s * weeks_from_start
        
        # Kan probleem worden by ReLu omdat slope negatief wordt door minimalisering Loss
        FVC_pred = weeks_from_start * slope + FVC_start

        ## ** Hier kan een fout komen doordat de afgeleide moeilijker te berekenen is
        sigma_clip = tf.maximum(tf.abs(sigma), 70)
        delta = tf.abs(y_true[:, 0] - FVC_pred)
        delta = tf.minimum(delta, 1000)
        ## **
        
        sq2 = tf.sqrt(tf.dtypes.cast(2, dtype=tf.float32))
        loss = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip * sq2)
        return K.mean(loss)
    
    loss = score
    opt = tf.keras.optimizers.Adam(1e-5)
    
    model.compile(optimizer=opt, loss = Laplace_log_likelihood)#, loss_weights = [...])

    return model